# From raw image to Cube

In [23]:
import os, glob
import cv2
import numpy as np
from facenet_pytorch import MTCNN

# mtcnn = MTCNN(post_process=False, device="cuda:0")
mtcnn = MTCNN(post_process=False, device="cpu")

In [24]:
new_size = (128, 128)


def mtcnn_landmark(img, boxes, points):
    # order sort
    boxes_resz = boxes[0].astype(int)
    boxes_resz[:, 2] = boxes_resz[:, 2] - boxes_resz[:, 0]
    boxes_resz[:, 3] = boxes_resz[:, 3] - boxes_resz[:, 1]
    width, height = np.mean(boxes_resz[:, 2]), np.mean(boxes_resz[:, 3])

    points_resz = points[0].astype(np.float32)
    points_resz = sorted(points_resz, key=lambda pts: pts[0][1])
    points_resz[0:6] = sorted(points_resz[0:6], key=lambda pts: pts[0][0])
    points_resz[6:12] = sorted(points_resz[6:12], key=lambda pts: pts[0][0])
    points_resz[12:18] = sorted(points_resz[12:18], key=lambda pts: pts[0][0])
    points_resz[18:24] = sorted(points_resz[18:24], key=lambda pts: pts[0][0])
    points_resz[24:30] = sorted(points_resz[24:30], key=lambda pts: pts[0][0])
    points_resz[30:36] = sorted(points_resz[30:36], key=lambda pts: pts[0][0])
    points_resz.reverse()

    pts_zero = points_resz[0]
    pts_zero_c = np.mean(pts_zero, axis=0)
    left_top = np.array([pts_zero_c[0] - width / 2, pts_zero_c[1] - height / 2])

    # crop and save
    img_gray = cv2.cvtColor(img[0], cv2.COLOR_BGR2GRAY)
    st = []
    for i in range(len(points_resz)):

        pts = points_resz[i]
        pts_c = np.mean(pts, axis=0)
        left_top_new = left_top + (pts_c - pts_zero_c)

        box = np.array([*left_top_new, left_top_new[0] + width, left_top_new[1] + height], dtype=int)

        img_gray_cropped = img_gray[max(0, box[1]) : max(0, box[3]), max(0, box[0]) : max(0, box[2])]
        st.append(cv2.resize(img_gray_cropped, new_size))

    return np.stack(st).transpose(1, 2, 0)

In [25]:
import scipy.io as io

# path_base = "F:/gist-grad/research/projects/hsi-face-recog/20240724-demo/"
path_base = "C:/Users/youngin/Desktop/2025_project/python/dataset/01-yi_choi"

for file in glob.glob("**/*.bmp", root_dir=path_base):

    img = cv2.imread(os.path.join(path_base, file))
    H, W, C = img.shape
    img = img.reshape(1, H, W, C)
    boxes, probs, points = mtcnn.detect(img, landmarks=True)
    cube = mtcnn_landmark(img, boxes, points)
    print(file, cube.shape)

    io.savemat(os.path.join(path_base, ".".join([*file.split(".")[:-1], "mat"])), {"cube_meas": cube})



01-live\01-live.bmp (128, 128, 36)
02-paper\02-print.bmp (128, 128, 36)
03-iphone\03-iphone.bmp (128, 128, 36)
